## Normalization

In [25]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import normalize
from sklearn.preprocessing import Normalizer

iris = load_iris()
data = pd.DataFrame(iris.data, columns=iris.feature_names)

# ------ toy ------
def normalize_l1(sample):
    """
    :param sample: DataFrame 的一行(axis=1) , 一个 Series
    :return: 经过 l1 正则的样本 , 一个 Series
    """
    return sample/sum(abs(sample))

print(data.head().apply(normalize_l1, axis=1))  # 对每一个样本操作
print('------------------------------------')

# ------ normalize function ------
print(pd.DataFrame(normalize(data, norm='l1'), columns=iris.feature_names).head())
print('------------------------------------')

# ------ Normalizer ------
scaler = Normalizer(norm='l1').fit(data)
print(pd.DataFrame(scaler.transform(data), columns=iris.feature_names).head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0           0.500000          0.343137           0.137255          0.019608
1           0.515789          0.315789           0.147368          0.021053
2           0.500000          0.340426           0.138298          0.021277
3           0.489362          0.329787           0.159574          0.021277
4           0.490196          0.352941           0.137255          0.019608
------------------------------------
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0           0.500000          0.343137           0.137255          0.019608
1           0.515789          0.315789           0.147368          0.021053
2           0.500000          0.340426           0.138298          0.021277
3           0.489362          0.329787           0.159574          0.021277
4           0.490196          0.352941           0.137255          0.019608
------------------------------------
   sepal lengt